In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,26636
2,Huelva,Confirmados PDIA 14 días,1237
3,Huelva,Tasa PDIA 14 días,"241,0507239316406"
4,Huelva,Confirmados PDIA 7 días,684
5,Huelva,Total Confirmados,26823
6,Huelva,Curados,22204
7,Huelva,Fallecidos,357


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  26636.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6946.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 922 personas en los últimos 7 días 

Un positivo PCR cada 545 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,26636.0,684.0,1237.0,513170.0,133.289163,241.050724,134.0
Condado-Campiña,7986.0,337.0,668.0,156231.0,215.706230,427.571993,64.0
Huelva-Costa,15839.0,291.0,475.0,289548.0,100.501471,164.048793,50.0
Huelva (capital),6946.0,156.0,264.0,143837.0,108.456100,183.541092,26.0
Almonte,1189.0,108.0,204.0,24507.0,440.690415,832.415228,25.0
Sierra de Huelva-Andévalo Central,2385.0,42.0,67.0,67391.0,62.322862,99.419804,14.0
Cartaya,1116.0,81.0,116.0,20083.0,403.326196,577.602948,11.0
Rociana del Condado,427.0,39.0,72.0,7939.0,491.245749,906.915229,10.0
Lucena del Puerto,181.0,17.0,19.0,3261.0,521.312481,582.643361,8.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
San Bartolomé de la Torre,266.0,17.0,43.0,3761.0,452.007445,1143.312949,0.0
Rociana del Condado,427.0,39.0,72.0,7939.0,491.245749,906.915229,10.0
Almonte,1189.0,108.0,204.0,24507.0,440.690415,832.415228,25.0
Sanlúcar de Guadiana,6.0,3.0,3.0,403.0,744.416873,744.416873,0.0
Bonares,218.0,19.0,36.0,6060.0,313.531353,594.059406,0.0
Lucena del Puerto,181.0,17.0,19.0,3261.0,521.312481,582.643361,8.0
Cartaya,1116.0,81.0,116.0,20083.0,403.326196,577.602948,11.0
Condado-Campiña,7986.0,337.0,668.0,156231.0,215.706230,427.571993,64.0
Moguer,963.0,57.0,84.0,21867.0,260.666758,384.140486,8.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Bollullos Par del Condado,709.0,10.0,53.0,14387.0,69.507194,368.388128,3.0,0.188679
Gibraleón,555.0,3.0,15.0,12737.0,23.553427,117.767135,0.0,0.200000
Trigueros,245.0,4.0,17.0,7862.0,50.877639,216.229967,1.0,0.235294
Paterna del Campo,123.0,1.0,4.0,3457.0,28.926815,115.707261,0.0,0.250000
Isla Cristina,2014.0,1.0,4.0,21393.0,4.674426,18.697705,0.0,0.250000
Ayamonte,1252.0,2.0,7.0,21104.0,9.476876,33.169067,1.0,0.285714
Villalba del Alcor,469.0,3.0,9.0,3366.0,89.126560,267.379679,0.0,0.333333
Lepe,2036.0,6.0,17.0,27880.0,21.520803,60.975610,1.0,0.352941
San Bartolomé de la Torre,266.0,17.0,43.0,3761.0,452.007445,1143.312949,0.0,0.395349
